# Introdução

Depósitos a prazo são importantíssimos para os bancos. Dentre as várias abordagens para vendê-los, o marketing por telefone é um dos métodos mais eficientes. Entretanto, esse tipo de divulgação é cara, uma vez que requer grande investimento em call-centers.

O objetivo desse programa é identificar os clientes com maior potencial para se inscrever em um depósito a prazo, direcionando as ligações telefônicas para eles.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import random

dadosclientes = pd.read_csv('../input/banking-dataset-marketing-targets/train.csv',sep = ';', encoding = 'utf-8',engine='c')
teste = pd.read_csv('../input/banking-dataset-marketing-targets/test.csv',sep = ';', encoding = 'utf-8',engine='c')
# O dataframe estava com apenas 1 coluna, com os valores separados por ";". Foi necessário separar nas 17 colunas.
dadosclientes


# Agora vamos tratar os dados !


## Dados:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
"blue-collar","self-employed","retired","technician","services")

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

6 - balance: average yearly balance, in euros (numeric)

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")

9 - contact: contact communication type (categorical: "unknown","telephone","cellular")

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)

15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")


Output variable (desired target):
17 - y - has the client subscribed a term deposit? (binary: "yes","no")


Missing Attribute Values: None

'''

**A princípio, todos os dados são importantes. Nossa missão é transformar as strings em números, para poder aplicar o modelo.**


## Substituindo os valores "unknown"

In [ ]:
dadosclientes["job"].value_counts()

Para substituirmos os valores 'unknown', vamos ver os empregos mais recorrentes com base na educação, a fim de ter um modelo mais preciso.

In [ ]:
#Vamos montar um dataframe de proporção educação/emprego
eduRatio = pd.DataFrame({'Job' : []})
for i in dadosclientes["job"].unique():
    eduRatio = eduRatio.append(dadosclientes[(dadosclientes["job"] == i)]["education"].value_counts().to_frame().iloc[0] * 100 / dadosclientes[(dadosclientes["job"] == i)]["education"].value_counts().sum())
eduRatio["Job"] = dadosclientes["job"].unique()
eduRatio

In [ ]:
#agora podemos trocar os valores 'unknown' das profissões pela profissão mais recorrente no nível de escolaridade
dadosclientes.loc[(dadosclientes.job == "unknown") & (dadosclientes.education == "secondary"),"job"] = "services"
dadosclientes.loc[(dadosclientes.job == "unknown") & (dadosclientes.education == "primary"),"job"] = "housemaid"
dadosclientes.loc[(dadosclientes.job == "unknown") & (dadosclientes.education == "tertiary"),"job"] = "management"
dadosclientes.loc[(dadosclientes.job == "unknown"),"job"] = "blue-collar" #profissão mais recorrente

In [ ]:
dadosclientes["job"].value_counts()

Show! fazendo o mesmo para o nível de escolaridade:


In [ ]:
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "admin."),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "management"),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "services"),"education"] = "tertiary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "technician."),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "retired"),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "blue-collar"),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "housemaid."),"education"] = "primary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "self-employed"),"education"] = "tertiary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "student"),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "entrepreneur"),"education"] = "tertiary"
dadosclientes.loc[(dadosclientes.education == "unknown") & (dadosclientes.job == "unemployed"),"education"] = "secondary"
dadosclientes.loc[(dadosclientes.education == "unknown"),"education"] = "secondary"

In [ ]:
dadosclientes["education"].value_counts()

In [ ]:
dadosclientes['job'] = dadosclientes['job'].map({"admin.":0,"unemployed":1,"management":2,"housemaid":3,"entrepreneur":4,"student":5, "blue-collar":6,"self-employed":7,"retired":8,"technician":9,"services":10})
dadosclientes['marital'] = dadosclientes['marital'].map({'married':0, 'divorced':1, 'single':2})
dadosclientes['education'] = dadosclientes['education'].map({"secondary":0,"primary":1,"tertiary":2})
dadosclientes['default'] = dadosclientes['default'].map({'no':0, 'yes':1})
dadosclientes['housing'] = dadosclientes['housing'].map({'no':0, 'yes':1})
dadosclientes['loan'] = dadosclientes['loan'].map({'no':0, 'yes':1})
dadosclientes['contact'] = dadosclientes['contact'].map({"unknown":0,"telephone":1,"cellular":2})
dadosclientes['month'] = dadosclientes['month'].map({'jan':0, 'feb':1, 'mar':2, 'apr':3, 'may':4, 'jun':5, 'jul':6, 'aug':7, 'sep':8, 'oct':9, 'nov':10, 'dec':11})
dadosclientes['poutcome'] = dadosclientes['poutcome'].map({"unknown":0,"other":1,"failure":2,"success":3})
dadosclientes['y'] = dadosclientes['y'].map({'no':0, 'yes':1})

dadosclientes

In [ ]:
dadosclientes['y'].value_counts()

# Aplicando o modelo

Beleza! Agora que os dados estão *ready to go*, podemos aplicar o nosso modelo.


## Qual modelo aplicar ?

Eu ainda não entendo muito da diferença entre os modelos que aprendi :p. Vou utilizar o *RandomForestClassifier* pois, pelo visto, ele é mais apropriado para quando os dados estão interligados. E é o caso desse problema, os dados interferem entre si. Por exemplo, a idade pode ser relevante, mas levando em conta outros fatores, como o emprego, o fator idade pode ter um peso ainda maior na decisão de tomar ou não o empréstimo. 
Outra vantagem, é poder ver a importância de cada feature na determinação da classificação. Vamos ter uma boa noção do perfil de pessoa que toma empréstimos a prazo.


In [ ]:
x = dadosclientes.drop(['y'], axis = 1)
y = dadosclientes['y']

In [ ]:


x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25) 
model = RandomForestClassifier()
model.fit(x_train,y_train)

result = model.predict(x_test)
print(accuracy_score(result,y_test))  # Porcentagem de acertos
print(confusion_matrix(result, y_test))

Caramba, que modelo bom, 90% de acerto! SQN.
Apesar de acertar muito quando o indíviduo não se inscreve em um depósito, quando é o caso da pessoa realmente querer tomar um empréstimo, o modelo erra mais do que acerta.

## Oversampling

Para corrigir o desbalanço do dataset, vamos aumentar a ocorrência de _yes_, isto é, de pessoas que pegam o impréstimo.
Utilizarei a técnica de oversampling SMOTE(Synthetic Minority Oversampling Technique) que, bem resumidamente, cria novos casos da classe minoritária criando novos vizinhos entre os k-vizinhos mais próximos de um ponto.

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

x_smote, y_smote = smote.fit_resample(x, y)

y_smote.value_counts()

Agora vamos treinar com o dataset balanceado.


In [ ]:
x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(x_smote,y_smote, test_size = 0.25, random_state = 10) 
modelo_balanceado = RandomForestClassifier()
modelo_balanceado.fit(x_train_smote,y_train_smote)

result = model.predict(x_test_smote)
print(accuracy_score(result,y_test_smote))  # Porcentagem de acertos
print(confusion_matrix(result, y_test_smote))

# Brincando de plotar

Vamos plotar alguns gráficos e árvores de decisão.


In [ ]:
import matplotlib.pyplot as matplotlib
import numpy

# Faz um plot das importâncias para um Random Forest
def plot_importances(model,df):
    importances = model.feature_importances_
    indices = numpy.argsort(importances)
    matplotlib.figure(figsize=(12,8))
    matplotlib.barh(range(len(indices)), importances[indices], color='b', align='center')
    matplotlib.yticks(range(len(indices)), df.columns[indices])
    matplotlib.suptitle('Importância das características')
    matplotlib.show()


In [ ]:
plot_importances(modelo_balanceado, dadosclientes)

## Interessante!
Parece que o fator mais importante na decisão de tomar ou não o empréstimo é a duração da última chamada telefônica do banco. Faz sentido, normalmente quem tem maior interesse tende a ficar mais tempo no telefone se informando/negociando sobre o depósito.

Entretanto, supondo que queremos ligar para clientes pela primeira vez, esse parâmetro se torna inútil. Vamos então fazer um modelo que não leva em conta essa e outras características que dependem de contato anterior ao cliente.



In [ ]:
dadosclientes2 = dadosclientes.drop(columns = ['duration','day', 'month','campaign','previous', 'pdays','poutcome'])

x2 = dadosclientes.drop(['y'], axis = 1)
y2 = dadosclientes['y']

x_smote2, y_smote2 = smote.fit_resample(x2, y2)

y_smote2.value_counts()

x_train_smote2, x_test_smote2, y_train_smote2, y_test_smote2 = train_test_split(x_smote2,y_smote2, test_size = 0.25, random_state = 10) 
modelo_balanceado2 = RandomForestClassifier()
modelo_balanceado2.fit(x_train_smote2,y_train_smote2)

resultado = modelo_balanceado2.predict(x_test_smote2)
print(accuracy_score(resultado,y_test_smote2))  # Porcentagem de acertos
print(confusion_matrix(resultado, y_test_smote2))

In [ ]:
dadosclientes2.head()

In [ ]:
plot_importances(modelo_balanceado2, dadosclientes2)